In [0]:
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple"\
  .split(" ")
words = spark.sparkContext.parallelize(myCollection, 2)


In [0]:

words.map(lambda word: (word.lower(), 1))


Out[3]: PythonRDD[1] at RDD at PythonRDD.scala:58

keyBy
- specify function that creates the key from your current value


In this case, you are keying by the first letter in the word. Spark stores the record as the value for the keyed RDD:

In [0]:
keyword = words.keyBy(lambda word: word.lower()[0])


mapValues
- you have a set of key–value pairs, you can begin manipulating them as such
- If a tuple, Spark will assume that the first element is the key, and the second is the value
- can explicitly choose to map-over the values (and ignore the individual keys)

In [0]:

keyword.mapValues(lambda word: word.upper()).collect()


Out[4]: [('s', 'SPARK'),
 ('t', 'THE'),
 ('d', 'DEFINITIVE'),
 ('g', 'GUIDE'),
 (':', ':'),
 ('b', 'BIG'),
 ('d', 'DATA'),
 ('p', 'PROCESSING'),
 ('m', 'MADE'),
 ('s', 'SIMPLE')]

flatMap over the rows 
- to expand the number of rows that you have to make
- so that each row represents a character

Each character as we converted them into array:

In [0]:
keyword.flatMapValues(lambda word: word.upper()).collect()


Out[5]: [('s', 'S'),
 ('s', 'P'),
 ('s', 'A'),
 ('s', 'R'),
 ('s', 'K'),
 ('t', 'T'),
 ('t', 'H'),
 ('t', 'E'),
 ('d', 'D'),
 ('d', 'E'),
 ('d', 'F'),
 ('d', 'I'),
 ('d', 'N'),
 ('d', 'I'),
 ('d', 'T'),
 ('d', 'I'),
 ('d', 'V'),
 ('d', 'E'),
 ('g', 'G'),
 ('g', 'U'),
 ('g', 'I'),
 ('g', 'D'),
 ('g', 'E'),
 (':', ':'),
 ('b', 'B'),
 ('b', 'I'),
 ('b', 'G'),
 ('d', 'D'),
 ('d', 'A'),
 ('d', 'T'),
 ('d', 'A'),
 ('p', 'P'),
 ('p', 'R'),
 ('p', 'O'),
 ('p', 'C'),
 ('p', 'E'),
 ('p', 'S'),
 ('p', 'S'),
 ('p', 'I'),
 ('p', 'N'),
 ('p', 'G'),
 ('m', 'M'),
 ('m', 'A'),
 ('m', 'D'),
 ('m', 'E'),
 ('s', 'S'),
 ('s', 'I'),
 ('s', 'M'),
 ('s', 'P'),
 ('s', 'L'),
 ('s', 'E')]

In [0]:
# in the key–value pair format, we can also extract the specific keys or values:
keyword.keys().collect()
keyword.values().collect()


Sample an RDD by a set of keys
- via an approximation or exactly 
- both operations can do so with or without replacement,  or sampling by a fraction by a given key
- This is done via simple random sampling with one pass over the RDD, which produces a sample of size that’s approximately equal to the sum of math.ceil(numItems *samplingRate) over all key values
- differs from sampleByKey in that you make additional passes over the RDD toc reate a sample size that’s exactly equal to the sum of math.ceil(numItems * samplingRate) over all key values with a 99.99% confidence
- When sampling without replacement, you need one additional pass over the RDD to guarantee sample size; when sampling with replacement, you need two additional passes

In [0]:
import random
distinctChars = words.flatMap(lambda word: list(word.lower())).distinct()\
  .collect()
sampleMap = dict(map(lambda c: (c, random.random()), distinctChars))
words.map(lambda word: (word.lower()[0], word))\
  .sampleByKey(True, sampleMap, 6).collect()


Out[8]: [('t', 'The'),
 ('t', 'The'),
 ('d', 'Definitive'),
 ('d', 'Data'),
 ('s', 'Simple'),
 ('s', 'Simple')]

In [0]:
# Can perform aggregations on plain RDDs or on PairRDDs, depending on the method that you are using: 

chars = words.flatMap(lambda word: word.lower())
KVcharacters = chars.map(lambda letter: (letter, 1))
def maxFunc(left, right):
  return max(left, right)
def addFunc(left, right):
  return left + right
nums = sc.parallelize(range(1,31), 5)

# with this, you can implement something like countByKey, which counts the items per eachkey


In [0]:
KVcharacters.countByKey()


Out[11]: defaultdict(int,
 {'s': 4,
 'p': 3,
 'a': 4,
 'r': 2,
 'k': 1,
 't': 3,
 'h': 1,
 'e': 7,
 'd': 4,
 'f': 1,
 'i': 7,
 'n': 2,
 'v': 1,
 'g': 3,
 'u': 1,
 ':': 1,
 'b': 1,
 'o': 1,
 'c': 1,
 'm': 2,
 'l': 1})

In [0]:

import functools

KVcharacters.groupByKey().map(lambda row: (row[0], reduce(addFunc, row[1])))\
  .collect()
# note this is Python 2, reduce must be imported from functools in Python 3


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1762652494941869> in <module> 
 1 import functools
 2 
 ----> 3 KVcharacters . groupByKey ( ) . map ( lambda row : ( row [ 0 ] , reduce ( addFunc , row [ 1 ] ) ) ) \ 
 4 . collect ( ) 
 5 # note this is Python 2, reduce must be imported from functools in Python 3 

 /databricks/spark/python/pyspark/rdd.py in collect (self) 
 901 # Default path used in OSS Spark / for non-credential passthrough clusters: 
 902 with SCCallSiteSync ( self . context ) as css : 
 --> 903 sock_info = self . ctx . _jvm . PythonRDD . collectAndServe ( self . _jrdd . rdd ( ) ) 
 904 return list ( _load_from_socket ( sock_info , self . _jrdd_deserializer ) ) 
 905 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 21.0 failed 1 times, most recent failure: Lost task 1.0 in stage 21.0 (TID 43, ip-10-172-244-18.us-west-2.compute.internal, executor driver): org.apache.spark.api.python.PythonException: 'NameError: name 'reduce' is not defined', from <command-1762652494941869>, line 3. Full traceback below:
Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 654, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 646, in process
 serializer.dump_stream(out_iter, outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 279, in dump_stream
 vs = list(itertools.islice(iterator, batch))
 File "/databricks/spark/python/pyspark/util.py", line 109, in wrapper
 return f(*args, **kwargs)
 File "<command-1762652494941869>", line 3, in <lambda>
NameError: name 'reduce' is not defined

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:598)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:733)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:716)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:551)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at org.apache.spark.InterruptibleIterator.foreach(InterruptibleIterator.scala:28)
	at scala.collection.generic.Growable.$plus$plus$eq(Growable.scala:62)
	at scala.collection.generic.Growable.$plus$plus$eq$(Growable.scala:53)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:105)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:49)
	at scala.collection.TraversableOnce.to(TraversableOnce.scala:315)
	at scala.collection.TraversableOnce.to$(TraversableOnce.scala:313)
	at org.apache.spark.InterruptibleIterator.to(InterruptibleIterator.scala:28)
	at scala.collection.TraversableOnce.toBuffer(TraversableOnce.scala:307)
	at scala.collection.TraversableOnce.toBuffer$(TraversableOnce.scala:307)
	at 

reduceByKey
- more stable approach is to perform a flatMap
- then perform a map, to map each letter instance to the number one, and then perform a reduceByKey with a summation function in order to collect back the array
- implementation is much more stable because the reduce happens within each partition and doesn’t need to put everything in memory
- no incurred shuffle during this operation; everything happens at each worker individually before performing a final reduce.
- greatly enhances the speed at which you can perform the operation as well as the stability of the operation

In [0]:
KVcharacters.reduceByKey(addFunc).collect()

Out[20]: [('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

aggregate
-  function requires a null and start value and then requires you to specify two different functions
- first aggregates within partitions, the second aggregates across partitions
- start value will be used at both aggregation levels
- does have some performance implications because it performs the final aggregation on the driver 
- If the results from the executors are too large, they can take down the driver with an OutOfMemoryError

In [0]:
nums.aggregate(0, maxFunc, addFunc)


Out[21]: 90

treeAggregate 
- does the same thing as aggregate (at the user level) but does so in a different way
- basically “pushes down” some of the subaggregations (creating a tree from executor to executor) before performing the final aggregation on the driver
- multiple levels can help you to ensure that the driver does not run out of memory in the process of the aggregation
- these tree-based implementations are often to try to improve stability in certain operations

In [0]:
depth = 3
nums.treeAggregate(0, maxFunc, addFunc, depth)


Out[22]: 90

In [0]:
# aggregateByKey
# - This function does the same as aggregate but instead of doing it partition by partition, it does it by key

KVcharacters.aggregateByKey(0, addFunc, maxFunc).collect()


Out[23]: [('s', 3),
 ('p', 2),
 ('r', 1),
 ('h', 1),
 ('d', 2),
 ('i', 4),
 ('g', 2),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 3),
 ('k', 1),
 ('t', 2),
 ('e', 4),
 ('f', 1),
 ('n', 1),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

combineByKey
- Instead of specifying an aggregation function, you can specify a combiner
- combiner operates on a given key and merges the values according to some function
- then goes to merge the different outputs of the combiners to give us our result
- can specify the number of output partitions as a custom output partitioner as well

In [0]:
def valToCombiner(value):
  return [value]
def mergeValuesFunc(vals, valToAppend):
  vals.append(valToAppend)
  return vals
def mergeCombinerFunc(vals1, vals2):
  return vals1 + vals2
outputPartitions = 6
KVcharacters\
  .combineByKey(
    valToCombiner,
    mergeValuesFunc,
    mergeCombinerFunc,
    outputPartitions)\
  .collect()


Out[24]: [('s', [1, 1, 1, 1]),
 ('d', [1, 1, 1, 1]),
 ('l', [1]),
 ('v', [1]),
 (':', [1]),
 ('p', [1, 1, 1]),
 ('r', [1, 1]),
 ('c', [1]),
 ('k', [1]),
 ('t', [1, 1, 1]),
 ('n', [1, 1]),
 ('u', [1]),
 ('o', [1]),
 ('h', [1]),
 ('i', [1, 1, 1, 1, 1, 1, 1]),
 ('g', [1, 1, 1]),
 ('b', [1]),
 ('a', [1, 1, 1, 1]),
 ('e', [1, 1, 1, 1, 1, 1, 1]),
 ('f', [1]),
 ('m', [1, 1])]

foldByKey
-  merges the values for each key using an associative function and a neutral “zero value"
- can be added to the result an arbitrary number of times
-  must not change the result (e.g., 0 for addition, or 1 for multiplication)

In [0]:

KVcharacters.foldByKey(0, addFunc).collect()


Out[25]: [('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

CoGroups
-  give you the ability to group together up to three key–value RDDs together in Scala and two in Python
- joins the given values by key
- effectively just a group-based joinon an RDD
-  can also specify a number of output partitions or a custom partitioning function to control exactly how this data is distributed across the cluster

In [0]:
import random
distinctChars = words.flatMap(lambda word: word.lower()).distinct()
charRDD = distinctChars.map(lambda c: (c, random.random()))
charRDD2 = distinctChars.map(lambda c: (c, random.random()))
charRDD.cogroup(charRDD2).take(5)


Out[26]: [('s',
 (<pyspark.resultiterable.ResultIterable at 0x7fb171626f90>,
 <pyspark.resultiterable.ResultIterable at 0x7fb171626490>)),
 ('p',
 (<pyspark.resultiterable.ResultIterable at 0x7fb171626c90>,
 <pyspark.resultiterable.ResultIterable at 0x7fb171626590>)),
 ('r',
 (<pyspark.resultiterable.ResultIterable at 0x7fb17168b910>,
 <pyspark.resultiterable.ResultIterable at 0x7fb17168b610>)),
 ('i',
 (<pyspark.resultiterable.ResultIterable at 0x7fb1715da310>,
 <pyspark.resultiterable.ResultIterable at 0x7fb1786cb650>)),
 ('g',
 (<pyspark.resultiterable.ResultIterable at 0x7fb171630f10>,
 <pyspark.resultiterable.ResultIterable at 0x7fb1716302d0>))]

joins
- RDDs have much the same joins as we saw in the Structured API
- although RDDs are much more involved for you
- follow the same basic format: 
	- the two RDDs we would like to join
	- optionally, either - 
		-  the number of output partitions 
		- or the customer partition function to which they should output

In [0]:
keyedChars = distinctChars.map(lambda c: (c, random.random()))
outputPartitions = 10
KVcharacters.join(keyedChars).count()
KVcharacters.join(keyedChars, outputPartitions).count()


Zips
- combine two RDDs, so it’s worth labeling it as a join
- zip allows you to “zip” together two RDDs, assuming that they have the same length
- creates a PairRDD
- The two RDDs must have the same number of partitions as well as the same number of elements

In [0]:
numRange = sc.parallelize(range(10), 2)
words.zip(numRange).collect()


Out[27]: [('Spark', 0),
 ('The', 1),
 ('Definitive', 2),
 ('Guide', 3),
 (':', 4),
 ('Big', 5),
 ('Data', 6),
 ('Processing', 7),
 ('Made', 8),
 ('Simple', 9)]

coalesce
-  effectively collapses partitions on the same worker in order to avoid a shuffle of the data when repartitioning
ex.  if our words RDD is currently two partitions, we can collapse that to one partition by using coalesce without bringing about a shuffle of the data:

In [0]:
words.coalesce(1).getNumPartitions() # 1


Out[29]: 1

In [0]:
words.repartition(10)


Out[32]: MapPartitionsRDD[111] at coalesce at NativeMethodAccessorImpl.java:0

Custom Partitioning
- one of the primary reasons you’d want to use RDDs
- Custom partitioners are not available in the Structured APIs because they don’t really have a logical counterpart
- They’re a low-level, implementation detail that can have a significant effect on whether your jobs run successfully
- sole goal of custom partitioning is to even out the distribution of your data across the cluster so that you can work around problems like data skew
- If you’re going to use custom partitioners, you should drop down to RDDs from the StructuredAPIs, apply your custom partitioner, and then convert it back to a DataFrame or Dataset
- get the best of both worlds, only dropping down to custom partitioning when you need to
- To perform custom partitioning you need to implement your own class that extends Partitioner. 
- only when you have lots of domain knowledge about your problem space 
- if you’re just looking to partition on a value or even a set of values (columns), it’s worth just doing it in the DataFrame API

In [0]:
df = spark.read.option("header", "true").option("inferSchema", "true")\
  .csv("/FileStore/tables/online_retail_dataset.csv")
rdd = df.coalesce(10).rdd

df.printSchema()


root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: integer (nullable = true)
-- InvoiceDate: string (nullable = true)
-- UnitPrice: double (nullable = true)
-- CustomerID: integer (nullable = true)
-- Country: string (nullable = true)

Spark has two built-in Partitioners that you can leverage off in the RDD API
- HashPartitioner for discrete values 
- RangePartitioner - continuous values
- Spark’s Structured APIs already use these - but we can use the same thing in RDDs
- hash and range partitioners are useful but rudimentary
- At times, you will need to perform some very low-level partitioning because you’re working with very large data and large key skew 
- Key skew simply means that some keys have many, many more values than other keys
- You want to break these keys as much as possible to improve parallelism and prevent OutOfMemoryErrors during the course of execution
- Might be that you need to partition more keys if and only if the key matches a certain format. 

Ex. 
- might know that there are two customers in your dataset that always crash your analysis and we need to break them up further than other customer IDs
- these two are so skewed that they need to be operated on alone 
- whereas all of the others can be lumped into large groups

In [0]:
def partitionFunc(key):
  import random
  if key == 17850 or key == 12583:
    return 0
  else:
    return random.randint(1,2)

keyedRDD = rdd.keyBy(lambda row: row[6])
keyedRDD\
  .partitionBy(3, partitionFunc)\
  .map(lambda x: x[0])\
  .glom()\
  .map(lambda x: len(set(x)))\
  .take(5)


Out[34]: [2, 4298, 4311]